In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_BASE = os.environ.get("OPENAI_API_BASE")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
TAVILY_API_KEY = os.environ.get("TAVILY_API_KEY")

In [2]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
model = ChatOpenAI(model="qwen-max")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="你好我是小明！我住在合肥")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我的住的地方天气怎么样？")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='你好，小明！很高兴认识你。合肥是一个历史悠久的城市，也是安徽省的省会。有什么我可以帮助你的吗？或者，你有什么关于合肥的好去处想要分享？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 213, 'total_tokens': 251, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9b7de2fc-3ad3-4cbb-829f-4547b98afd42-0', usage_metadata={'input_tokens': 213, 'output_tokens': 38, 'total_tokens': 251, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_6b4b72882f1e427e9fcb44', 'function': {'arguments': '{"query": "合肥 天气"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 267, 'total_tokens': 290, 'completion_tokens_details': N

# Define Tool

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_result = search.invoke("合肥今天的天气怎么样？")

tools = [search]

In [4]:
for result in search_result:
    print(result)

{'url': 'https://tianqi.moji.com/weather/china/anhui/hefei', 'content': '合肥市今天实况：17度雾，湿度：89%，西北风：2级。白天：19度,雾。 夜间：多云，11度，天气偏凉了，墨迹天气建议您穿上厚些的外套或是保暖的羊毛衫，年老体弱者可以选择保暖的摇'}
{'url': 'https://tianqi.fliggy.com/340100/', 'content': '合肥11月10日00:01天气预报，今天11℃-20℃，小雨, 风向情况：东风，总降水量：0.00mm，相对湿度：91%。合肥今日生活指数: 交通指数，良好（天气较好，路面干燥，交通'}


# Using Language Models

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="qwen-max")
model_with_tools = model.bind_tools(tools)

In [6]:
response = model_with_tools.invoke([HumanMessage(content="你好！")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 你好！有什么我能帮助你的吗？
ToolCalls: []


In [7]:
response = model_with_tools.invoke([HumanMessage(content="合肥今天的天气怎么样？")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': '合肥今天天气'}, 'id': 'call_8e2767c581e34907a41925', 'type': 'tool_call'}]


# Create the agent

In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [9]:
response = agent_executor.invoke({"messages": [HumanMessage(content="你好！")]})

response["messages"]

[HumanMessage(content='你好！', additional_kwargs={}, response_metadata={}, id='3152c960-3933-45cd-91bc-cd835914773e'),
 AIMessage(content='你好！有什么我能帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 207, 'total_tokens': 215, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6091a88f-0dc5-4bb8-9d52-a2725eacc4bf-0', usage_metadata={'input_tokens': 207, 'output_tokens': 8, 'total_tokens': 215, 'input_token_details': {}, 'output_token_details': {}})]

In [10]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="合肥今天的天气怎么样？")]}
)
response["messages"]

[HumanMessage(content='合肥今天的天气怎么样？', additional_kwargs={}, response_metadata={}, id='39ca456a-51c2-4f59-a967-cf92ac1ef305'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0e64de36078b4c5dbf60a9', 'function': {'arguments': '{"query": "合肥今天天气"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 210, 'total_tokens': 232, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-bc66aec4-1960-4e94-bebb-561e7ffd2816-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '合肥今天天气'}, 'id': 'call_0e64de36078b4c5dbf60a9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 210, 'output_tokens': 22, 'total_tokens': 232, 'input_token_details': {}, 'output_token_details': {}}),
 ToolMessage(content='[{"url": "https://t

# Streaming

In [11]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="合肥今天的天气怎么样？")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7e165c8d068849fc873f14', 'function': {'arguments': '{"query": "合肥今天天气"}', 'name': 'tavily_search_results_json'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 210, 'total_tokens': 232, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-134805a6-eb53-4dff-b411-14ae9c589f75-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '合肥今天天气'}, 'id': 'call_7e165c8d068849fc873f14', 'type': 'tool_call'}], usage_metadata={'input_tokens': 210, 'output_tokens': 22, 'total_tokens': 232, 'input_token_details': {}, 'output_token_details': {}})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://tianqi.moji.com/weather/china/anhui/hefei", "content": "合肥市今天实况：17度雾，湿度：8

# Adding in memory

In [13]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="你好，我是小明！")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='你好，小明！很高兴见到你。有什么我可以帮助你的吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 211, 'total_tokens': 226, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-56c99002-57e8-4b30-a187-395efa7ebd0d-0', usage_metadata={'input_tokens': 211, 'output_tokens': 15, 'total_tokens': 226, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我叫什么？")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='你叫小明。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 240, 'total_tokens': 245, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-14c52742-5971-497d-abae-c44217206514-0', usage_metadata={'input_tokens': 240, 'output_tokens': 5, 'total_tokens': 245, 'input_token_details': {}, 'output_token_details': {}})]}}
----


In [15]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我叫什么？")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='您好！您并没有告诉我您的名字，所以我无法知道您叫什么。您可以告诉我您的名字吗？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 209, 'total_tokens': 230, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-max', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-76d29dc3-063e-4be2-9c2c-40283945d778-0', usage_metadata={'input_tokens': 209, 'output_tokens': 21, 'total_tokens': 230, 'input_token_details': {}, 'output_token_details': {}})]}}
----
